In [ ]:
## @file lab02_detection.py
## @brief Detects the track line closest to the bottom of the video frame
## @details This script processes a raw video and overlays a green circle on the center of the dark line shown on the screen
## @author Yuriel Dimayacyac
## @date 2026-01-20

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
from IPython.display import clear_output
import time
import os

# troubleshooting
print(os.path.exists('/content/raw_video_feed.mp4'))

cap = cv2.VideoCapture('/content/raw_video_feed.mp4')

# check if successful

if not cap.isOpened():
  print("Error. Could not open video")

else:
  frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
  print("The video has", frame_count, "frames")

  current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
  print("I am currently on frame:",current_frame)

  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = cap.get(cv2.CAP_PROP_FPS)

  print(f"Resolution: {width} x {height}")

  bottom_row = height - 1
  print(f"We will start iteration from row {bottom_row}, (the bottom row)")

  # initialize output video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  out = cv2.VideoWriter('output.mp4',fourcc,fps,(width,height))


In [ ]:
## @brief Find the y-coordinate of the center of the line given a row index
## @param frame The array of one frame from the video
## @param row_index The y-coordinate of the line center
## @param threshold Value that defines line
## @return The x-coordinate of the line center
## @return T/F Line detected
def locate_circle_pos(frame, row_index, threshold):

  # initialize iterator
  i = 0
  line_start = 0
  line_end = 0

  # initialize booleans
  line_start_detected = False
  line_end_detected = False

  # find the start of the line in the row
  while not line_start_detected:
    pixel_brightness = frame[row_index, i]
    if pixel_brightness < threshold:
      line_start_detected = True
      line_start = i

    # iterate pixel
    i += 1

    # break if line never detected
    if (i == width-1):
      break

  # initialize iterator
  k = i

  # iterate through pixels on row to find end of line
  while not line_end_detected:
    pixel_brightness = frame[row_index,k]

    # is the end of the row detected?
    if pixel_brightness >= threshold:
      line_end_detected = True
      line_end = k

    k += 1

    # break if line never detected
    if (k >= width-1):
      break

  if line_start_detected and line_end_detected:
    average_centre_position = (line_start+line_end)/2
    return True, (average_centre_position)

  # loop is finished and no line was found
  return False, 0


In [ ]:
ret, frame = cap.read()

## @var line_threshold
## @brief The grayscale intensity threshold for a line (lower = darker)
line_threshold = 100

# frame count
count = 0

# iterate through all frames
while ret:
  if not ret:
    break

  #troubleshooting
  cv2.putText(frame, f"Frame: {count}", (50, 50),
              cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

  if cap.get(cv2.CAP_PROP_POS_FRAMES) % 50 == 0:
    print(f"Processing frame {cap.get(cv2.CAP_PROP_POS_FRAMES)}...", end="\r")


  # initialize lineFound state
  lineFound = False

  # convert frame to grayscale for ease of processing
  grayscale_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  # iterate through rows from bottom row to top
  for j in range(bottom_row, -1, -1):
    # if true, line found
    lineFound, i = locate_circle_pos(grayscale_frame, j, line_threshold)

    if lineFound:
      centre = int(i), int(j) - 10
      break

  # Render outside of for-loop
  if lineFound:
    cv2.circle(frame, centre, 10, (0,255,0),-1)

  out.write(frame)

  count += 1

  # update frame value
  ret, frame = cap.read()

In [ ]:
# finalize the video processing
cap.release()
out.release()
print("\nDone! Video saved as output.mp4")